<a href="https://colab.research.google.com/github/ryanhao1115/ML-for-Fraud-Detection/blob/main/4_1_activelearning_ordinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Active Learning with Ordinal encoding data
1. Ordinal encoding categorical fileds
2. Process the labeled fraud invoice
3. Sample data for active learning
4. Train model
5. Choose top 100 uncertain invoice
6. Get back the labeled data

In [64]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report, roc_auc_score

In [65]:
## import dataset 
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file = path + 'sales_clean.csv'
df = pd.read_csv(file)

In [66]:
df = df.drop(columns='Unnamed: 0')

In [88]:
df.describe()

,branch,invoice_no,prod_cla,qty,total_amt,sale_price,ship_qty,return,discount_app,list_price,inv_ship_days,fraud
count,228601.000000,2.286010e+05,228601.000000,228601.000000,2.286010e+05,228601.000000,228601.000000,228601.000000,228601.000000,228601.000000,228601.000000,228601.000000
mean,716373.299509,2.110400e+09,11020.276814,2.073206,5.395319e+03,3124.179679,2.012332,0.000822,0.999869,3530.768339,4.421481,0.004462
std,48447.146978,3.109950e+07,12.908625,11.449643,2.513897e+04,4540.858362,10.723446,0.028666,0.011455,5131.707878,9.610810,0.090721
min,72010.000000,1.103721e+09,9999.000000,1.000000,3.053000e+01,29.200000,0.000000,0.000000,0.000000,33.000000,-1.000000,0.000000
25%,720008.000000,2.111333e+09,11020.000000,1.000000,4.172400e+02,415.120000,1.000000,0.000000,1.000000,469.090000,1.000000,0.000000
50%,720013.000000,2.111360e+09,11020.000000,1.000000,2.920350e+03,2326.810000,1.000000,0.000000,1.000000,2629.300000,2.000000,0.000000
75%,720021.000000,2.111387e+09,11020.000000,1.000000,4.188730e+03,3662.410000,1.000000,0.000000,1.000000,4138.520000,5.000000,0.000000
max,720037.000000,2.111415e+09,11060.000000,1189.000000,2.051452e+06,50442.470000,1189.000000,1.000000,1.000000,57000.000000,239.000000,2.000000


## import label data and sampling

In [68]:
## import fraud data
file = path + 'frauds.csv'
df_fraud = pd.read_csv(file,header=None)
df_fraud.columns = ['invoice_no']

In [69]:
file = path + 'frauds2.csv'
df_fraud2 = pd.read_csv(file,header=None)
df_fraud2.columns = ['invoice_no']

In [ ]:
df_fraud

In [71]:
def label_fraud(df, df_fraud, df_fraud2):
  df['fraud'] = np.zeros(len(df))
  frauds_l = df_fraud['invoice_no'].to_list()
  frauds_l2 = df_fraud2['invoice_no'].to_list()
  df.loc[df['invoice_no'].isin(frauds_l),'fraud'] = 1
  df.loc[df['invoice_no'].isin(frauds_l2),'fraud'] = 2
  return df

In [72]:
df = label_fraud(df, df_fraud,df_fraud2)

In [89]:
df['fraud'].value_counts()

0.0    228014
2.0       433
1.0       154
Name: fraud, dtype: int64

## Resampling to address the imbalance class
Because fraud labels are given to invoice level, need to keep records of same invoice together. 

In [115]:
df_fraud = df[df['fraud'] == 1]
df_non_fraud = df[df['fraud'] == 0]
df_fraud2 = df[df['fraud'] == 2]

In [116]:
df_fraud2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433 entries, 19646 to 217825
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   distributor    433 non-null    object 
 1   sales          433 non-null    object 
 2   branch         433 non-null    object 
 3   inv_type       433 non-null    object 
 4   invoice_no     433 non-null    object 
 5   product_no     433 non-null    object 
 6   prod_cla       433 non-null    object 
 7   qty            433 non-null    int64  
 8   total_amt      433 non-null    float64
 9   sale_price     433 non-null    float64
 10  ship_qty       433 non-null    int64  
 11  cust_type      433 non-null    object 
 12  return         433 non-null    int64  
 13  discount_app   433 non-null    int64  
 14  list_price     433 non-null    float64
 15  inv_ship_days  433 non-null    int64  
 16  fraud          433 non-null    float64
dtypes: float64(4), int64(5), object(8)
memory usage

In [117]:
def sampling_inv(df_non_fraud):
  '''
  Random sampling from unlabled invoices.
  '''
  sample_inv = df_non_fraud['invoice_no'].unique()
  print(len(sample_inv))
  sample_inv1 = np.random.choice(sample_inv,size=1000,replace=False)
  sample_inv = np.setdiff1d(sample_inv,sample_inv1)
  print(len(sample_inv))
  sample_inv2 = np.random.choice(sample_inv,size=10000,replace=False)
  sample_inv = np.setdiff1d(sample_inv,sample_inv2)
  print(len(sample_inv))
  return sample_inv1, sample_inv2, sample_inv


In [118]:
sample_inv1, sample_inv2, sample_inv = sampling_inv(df_non_fraud)

30686
29686
19686


In [119]:
len(np.unique(sample_inv2))

10000

In [120]:
## Build training dataset
df_train = df_non_fraud[df_non_fraud['invoice_no'].isin(sample_inv1)]
df_train = pd.concat([df_train,df_fraud])
df_train = df_train.sort_index()

In [121]:
## Build eval dataset
df_eval = df_non_fraud[df_non_fraud['invoice_no'].isin(sample_inv2)]
df_eval = pd.concat([df_eval,df_fraud2])
df_eval.loc[df_eval['fraud'] == 2, 'fraud'] = 1 
df_eval = df_eval.sort_index()

In [122]:
len(df_train)

7418

In [123]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7418 entries, 262 to 228150
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   distributor    7418 non-null   object 
 1   sales          7418 non-null   object 
 2   branch         7418 non-null   object 
 3   inv_type       7418 non-null   object 
 4   invoice_no     7418 non-null   object 
 5   product_no     7418 non-null   object 
 6   prod_cla       7418 non-null   object 
 7   qty            7418 non-null   int64  
 8   total_amt      7418 non-null   float64
 9   sale_price     7418 non-null   float64
 10  ship_qty       7418 non-null   int64  
 11  cust_type      7418 non-null   object 
 12  return         7418 non-null   int64  
 13  discount_app   7418 non-null   int64  
 14  list_price     7418 non-null   float64
 15  inv_ship_days  7418 non-null   int64  
 16  fraud          7418 non-null   float64
dtypes: float64(4), int64(5), object(8)
memory usage:

## Encoding

In [124]:
cols = df.columns.to_list()
cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']

In [125]:
def field_encoding(df_train, df_eval):
  '''
  Ordinal encode categorical fields.
  '''
  cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']
  df_train[cols] = df_train[cols].astype('str') 
  df_eval[cols] = df_eval[cols].astype('str') 
  df1 = pd.concat([df_train, df_eval])
  enc = LabelEncoder()
  for f in cols:
    enc.fit(df1[f])
    df_train[f] = enc.transform(df_train[f])
    df_eval[f] = enc.transform(df_eval[f])
  return df_train, df_eval

In [126]:
X_train, X_eval = field_encoding(df_train, df_eval)

In [127]:
X_train.nunique()

distributor       333
sales             147
branch             28
inv_type            2
invoice_no       1011
product_no        458
prod_cla            7
qty                44
total_amt        1879
sale_price       1548
ship_qty           43
cust_type          20
return              1
discount_app        2
list_price       1271
inv_ship_days      35
fraud               2
dtype: int64

## Standardscale data

In [100]:
scaler = StandardScaler()

In [128]:
y_train = X_train['fraud'].values
X_train = X_train.drop(columns=['fraud'])

X_train = scaler.fit_transform(X_train)
y_train = y_train.reshape(len(y_train),1)

In [129]:
y_eval = X_eval['fraud'].values
X_eval = X_eval.drop(columns=['fraud'])

X_eval = scaler.transform(X_eval)
y_eval = y_eval.reshape(len(y_eval),1)

In [130]:
X_train[0]

array([ 0.34925379,  1.3282699 ,  1.39139481,  0.01161143, -1.76370248,
        1.32479988, -0.08662241, -0.10594321,  0.15073646,  1.04364105,
       -0.10152181,  0.01993016,  0.        ,  0.01161143,  1.04205654,
        0.12487154])

In [104]:
y_eval.shape

(74685, 1)

## Build model

In [105]:
# define the  model
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [106]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 1,089
Trainable params: 1,089
Non-trainable params: 0
_________________________________________________________________


## Training model

In [131]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
232/232 [==============================] - 1s 3ms/step - loss: 0.1560 - accuracy: 0.9802
Epoch 2/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0535 - accuracy: 0.9864
Epoch 3/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0349 - accuracy: 0.9891
Epoch 4/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0265 - accuracy: 0.9912
Epoch 5/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0238 - accuracy: 0.9922
Epoch 6/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0195 - accuracy: 0.9926
Epoch 7/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0174 - accuracy: 0.9947
Epoch 8/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0159 - accuracy: 0.9951
Epoch 9/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0142 - accuracy: 0.9961
Epoch 10/100
232/232 [==============================] - 1s 3ms/step - loss: 0.0135 - accura

In [132]:
loss,accuracy = model.evaluate(X_eval, y_eval)
print(loss, accuracy)

2330/2330 [==============================] - 5s 2ms/step - loss: 0.4330 - accuracy: 0.9893
0.4330098628997803 0.9892820715904236


## Predict on eval dataset for further labeling.

In [133]:
y_pred = model.predict(X_eval)
y_cls = (y_pred > 0.5).astype("int32")

In [60]:
(y_pred > 0.5).sum()

827

In [134]:
def eval_model(y_eval, y_cls):
  y1 = y_eval.reshape(-1)
  y2 = y_cls.reshape(-1)
  print(roc_auc_score(y1, y2))
  print(classification_report(y1,y2))


In [135]:
eval_model(y_eval, y_cls)

0.5698541169168007
              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99     74115
         1.0       0.13      0.15      0.14       433

    accuracy                           0.99     74548
   macro avg       0.56      0.57      0.57     74548
weighted avg       0.99      0.99      0.99     74548



In [136]:
def record_to_label(df_eval,y_pred):
  df_eval['predict'] = y_pred
  df_eval['mid_value'] = abs(df_eval['predict'] - 0.5)
  df_eval = df_eval.sort_values('mid_value')
  value_limit = df_eval.iloc[100,-1]
  df_record4label = df_eval[(df_eval['mid_value'] <= value_limit)|(df_eval['fraud'] == 1)]
  path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
  file_csv = path + 'for_label.csv'
  df_record4label.to_csv(file_csv)
  return True


In [137]:
record_to_label(df_eval,y_pred)

True